In [1]:
import sys
sys.path.append('..')  # allow this notebook to find equal-level directories
%pylab inline
from importing_modules import *
# pyomo.environ as oe, seaborn as sns, plotly.plotly as py, plotly.graph_objs as go
# from util.gjh_wrapper import gjh_solve, make_df, from vis import acres_bars, zL_bars

Populating the interactive namespace from numpy and matplotlib


## Create problem instance

In [2]:
# Load data for each set, parameter, etc. to define a problem instance
objwrapper = CostObj()
# lrsegs = ['N42071SL2_2410_2700']
lrsegs = ['N51133RL0_6450_0000']
data = objwrapper.load_data(savedata2file=False, lrsegs_list=lrsegs)

# ---- Set the tau target load ----
for k in data.tau:
    data.tau[k] = 12
    taustr = str(round(data.tau[k], 1))
    
# Create concrete problem instance using the separately defined optimization model
mdl = objwrapper.create_concrete(data=data)
# Print the target load reduction values
for l in mdl.LRSEGS:
    for p in mdl.PLTNTS:
        print('%s: %d' % (mdl.tau[l,p], mdl.tau[l,p].value))

# ---- Solver name ----
localsolver = True
solvername = 'ipopt'
# solvername = 'minos'

tau[N51133RL0_6450_0000,S]: 12
tau[N51133RL0_6450_0000,P]: 12
tau[N51133RL0_6450_0000,N]: 12


## Solve problem instance

In [3]:
from collections import OrderedDict

df_list = []
solution_objectives = OrderedDict()
for ii in range(1, 10):
    print(ii)
    
    # Reassign the target load values (tau)
    newtau = ii
    for k in data.tau:
        mdl.tau[k] = newtau
        print(mdl.tau[k].value)
        taustr = str(round(mdl.tau[k].value, 1))
        
    # Set names for saving output
    looptimestamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
    loopname = ''.join(['output/costobj_tau', taustr, '_', solvername, '_',
                           looptimestamp])
        
    # Solve The Model
    myobj = SolveAndParse(instance=mdl, data=data, localsolver=localsolver, solvername=solvername)
    merged_df = myobj.solve()
    print('\nObjective is: %d' % oe.value(mdl.Total_Cost))
    
    # Save this run's objective value in a list
    solution_objectives[mdl.tau[k].value] = oe.value(mdl.Total_Cost)
    merged_df['solution_objectives'] = oe.value(mdl.Total_Cost)
    
    # Label this run in the dataframe
    merged_df['tau'] = mdl.tau[k].value
    sorteddf_byacres = merged_df.sort_values(by='acres')
    
    
#     # ---- Make zL Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '_zL', .png']))

#     zL_bars(df=merged_df, instance=mdl,
#             savefig=True, savefilepathandname=savefilepathandname)
    
    
#     # ---- Make Acres Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '_x', .png']))

#     objstr = ''.join(['Objective is: ', str(round(mdl.Total_Cost(), 2))])
#     titlestr = '\n'.join([objstr, 'labels are (cost per unit, total bmp instance cost)'])

#     acres_bars(df=sorteddf_byacres, instance=mdl, titlestr=titlestr,
#                savefig=True, savefilepathandname=savefilepathandname)
    
#     # -- Gradient, Jacobian, Hessian
#     gjh_filename, g = gjh_solve(instance=mdl,
#                             keepfiles=True,
#                             amplenv=ampl,
#                             basegjhpath=os.getcwd())

#     g_df = make_df(instance=mdl, filterbydf=merged_df, g=g)

#     g_df = sorteddf_byacres.merge(g_df, how='left',
#                                   on=['bmpshortname', 'landriversegment', 'loadsource'],
#                                   sort=False)
    
#     # ---- Make gradient Figure ----
#     g_df_filtered = g_df
#     keystrs = [str([x, y]) for x, y in zip(g_df_filtered['bmpshortname'], g_df_filtered['loadsource'])]
#     # Make Figure
#     fig = plt.figure(figsize=(10, 4))
#     rects = plt.barh(y=keystrs, width=g_df_filtered['g'])
#     ax = plt.gca()

#     ax.set_position([0.3,0.1,0.5,0.8])
#     plt.savefig(os.path.join(projectpath, ''.join([loopname, '_g', .png']))))
    
    # Save all of the solutions in a list
    df_list.append(sorteddf_byacres)
    
# Save the results to a .csv file
alldfs = pd.concat(df_list, ignore_index=True)
alldfs['x'] = list(zip(alldfs.bmpshortname, alldfs.landriversegment, alldfs.loadsource, alldfs.totalannualizedcostperunit))
filenamestr = ''.join(['output/costobj_tausequence_alldfs', '_', solvername, '_',
                           datetime.now().strftime('%Y-%m-%d_%H%M%S'), '.csv'])
alldfs.to_csv(os.path.join(projectpath, filenamestr))

1
1
1
1
Solver log file: 'logfile_loadobjective.log'
Solver solution file: '/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmp34lkmpe4.pyomo.sol'
Solver problem files: ('/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmp34lkmpe4.pyomo.nl',)
Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     3609
Number of nonzeros in Lagrangian Hessian.............:     5575

Total number of 

   3  5.4455401e+03 6.01e-01 3.71e+01  -1.0 1.00e+01    -  9.88e-01 1.90e-01f  1
   4  7.0509631e+03 5.12e-01 3.24e+01  -1.0 7.52e+02    -  5.46e-01 1.99e-01f  1
   5  1.0317351e+04 2.52e-01 1.75e+01  -1.0 1.09e+03    -  6.46e-01 5.04e-01h  1
   6  1.1909489e+04 1.13e-01 1.64e+01  -1.0 3.76e+02    -  8.99e-01 5.36e-01h  1
   7  1.2440156e+04 5.09e-02 4.71e+01  -1.0 3.18e+02    -  1.00e+00 4.72e-01h  1
   8  1.2755154e+04 6.31e-03 6.91e+01  -1.0 2.54e+02    -  1.00e+00 6.13e-01h  1
   9  1.2826096e+04 2.91e-03 1.87e+02  -1.0 1.53e+02    -  1.00e+00 5.51e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.2871714e+04 1.03e-03 3.47e+02  -1.0 7.14e+01    -  1.00e+00 6.35e-01h  1
  11  1.2883486e+04 5.01e-04 1.16e+03  -1.0 2.61e+01    -  1.00e+00 5.27e-01h  1
  12  1.2891405e+04 1.72e-04 1.97e+03  -1.0 1.23e+01    -  1.00e+00 6.47e-01h  1
  13  1.2893355e+04 8.48e-05 6.95e+03  -1.0 4.36e+00    -  1.00e+00 5.21e-01h  1
  14  1.2894699e+04 2.89e-05

  21  2.5240535e+04 1.25e-07 4.03e+06  -1.0 4.04e-03    -  1.00e+00 4.45e-02f  5
  22  2.5240608e+04 0.00e+00 1.00e-06  -1.0 3.86e-03    -  1.00e+00 1.00e+00h  1
  23  1.3269386e+04 0.00e+00 5.44e+06  -8.6 5.44e+01    -  3.90e-01 9.99e-01f  1
  24  1.2344928e+04 0.00e+00 4.96e+06  -8.6 2.91e+01    -  8.92e-02 1.00e+00f  1
  25  1.2330283e+04 0.00e+00 1.80e+06  -8.6 7.82e+00    -  6.37e-01 1.00e+00f  1
  26  1.2031294e+04 0.00e+00 1.71e+06  -8.6 9.91e+00    -  5.07e-02 1.00e+00f  1
  27  1.2029035e+04 0.00e+00 3.22e+05  -8.6 2.23e+00    -  8.12e-01 1.00e+00f  1
  28  1.1974393e+04 0.00e+00 3.06e+05  -8.6 8.72e+00    -  4.89e-02 1.00e+00f  1
  29  1.1973899e+04 0.00e+00 7.64e+04  -8.6 2.79e+00    -  7.51e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.1973635e+04 0.00e+00 8.25e+03  -8.6 2.21e+00    -  8.92e-01 9.99e-01f  1
  31  1.1973583e+04 0.00e+00 2.40e+02  -8.6 3.05e+00    -  9.71e-01 1.00e+00f  1
  32  1.1973580e+04 0.00e+00

  35  1.4529065e+05 0.00e+00 7.00e+02  -8.6 3.06e+00    -  9.71e-01 9.99e-01f  1
  36  1.4529065e+05 0.00e+00 3.98e+01  -8.6 5.70e-01    -  9.43e-01 1.00e+00f  1
  37  1.4529064e+05 0.00e+00 2.82e+01  -8.6 1.70e-04    -  2.93e-01 1.00e+00f  1
  38  1.4529064e+05 0.00e+00 1.75e+01  -8.6 1.81e-07    -  3.80e-01 1.00e+00f  1
  39  1.4529064e+05 0.00e+00 4.13e+00  -8.6 1.47e-07    -  7.63e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  1.4529064e+05 0.00e+00 1.30e+00  -8.6 9.68e-08    -  6.84e-01 1.00e+00f  1
  41  1.4529064e+05 0.00e+00 2.84e-14  -8.6 5.59e-09    -  1.00e+00 1.00e+00f  1

Number of Iterations....: 41

                                   (scaled)                 (unscaled)
Objective...............:   1.0221643273964373e+03    1.4529064192899509e+05
Dual infeasibility......:   2.8421709430404007e-14    4.0398674627795115e-12
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity....


Objective is: 359202
6
6
6
6
    'pyomo.core.base.suffix.Suffix'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().
    (type=<class 'pyomo.core.base.suffix.Suffix'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.suffix.Suffix'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
Solver log file: 'logfile_loadobjective.log'
Solver solution file: '/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmprcexe5eb.pyomo.sol'
Solver problem files:

 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     3609
Number of nonzeros in Lagrangian Hessian.............:     5575

Total number of variables............................:     1339
                     variables with only lower bounds:     1339
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1110
        inequality constraints with only lower boun

   6  1.0415383e+06 2.68e-01 3.44e+02  -1.0 4.85e+02    -  6.61e-01 6.79e-01h  1
   7  1.0850523e+06 8.86e-02 9.65e+01  -1.0 1.53e+02    -  9.90e-01 6.69e-01h  1
   8  1.0921571e+06 5.91e-02 1.09e+02  -1.0 5.02e+01    -  1.00e+00 3.32e-01h  1
   9  1.1049023e+06 5.79e-03 3.34e+01  -1.0 3.35e+01    -  1.00e+00 8.97e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.1051719e+06 4.68e-03 1.74e+03  -1.0 3.45e+00    -  1.00e+00 1.81e-01h  1
  11  1.1062592e+06 3.63e-04 4.02e+02  -1.0 2.83e+00    -  1.00e+00 9.06e-01h  1
  12  1.1062790e+06 2.89e-04 2.27e+04  -1.0 2.66e-01    -  1.00e+00 1.72e-01h  1
  13  1.1063639e+06 5.14e-06 5.03e+03  -1.0 2.20e-01    -  1.00e+00 9.09e-01h  1
  14  1.1063641e+06 5.15e-06 3.42e+05  -1.0 2.01e-02    -  1.00e+00 2.16e-02f  4
  15  1.1063689e+06 2.58e-06 2.90e+05  -1.0 1.96e-02    -  1.00e+00 5.75e-01h  1
  16  1.1063724e+06 1.53e-08 2.01e+04  -1.0 8.34e-03    -  1.00e+00 9.80e-01h  1
  17  1.1063724e+06 3.91e-08

  25  1.4175761e+06 6.75e-08 6.67e+05  -1.0 2.16e-03    -  1.00e+00 8.01e-01h  1
  26  1.4175761e+06 5.76e-08 3.55e+06  -1.0 4.29e-04    -  1.00e+00 1.25e-01f  4
  27  1.4175762e+06 0.00e+00 1.00e-06  -1.0 3.76e-04    -  1.00e+00 1.00e+00h  1
  28  1.4055348e+06 0.00e+00 5.88e+06  -8.6 5.14e+01    -  3.95e-01 9.99e-01f  1
  29  1.4046168e+06 0.00e+00 5.38e+06  -8.6 1.25e+01    -  8.64e-02 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.4046050e+06 0.00e+00 1.96e+06  -8.6 2.59e+00    -  6.36e-01 1.00e+00f  1
  31  1.4043086e+06 0.00e+00 1.86e+06  -8.6 4.31e+00    -  5.11e-02 1.00e+00f  1
  32  1.4043057e+06 0.00e+00 3.56e+05  -8.6 2.19e+00    -  8.08e-01 1.00e+00f  1
  33  1.4042763e+06 0.00e+00 3.45e+05  -8.6 1.98e+00    -  3.31e-02 9.94e-01f  1
  34  1.4042520e+06 0.00e+00 5.17e+04  -8.6 3.52e-01    -  8.50e-01 9.96e-01f  1
  35  1.4042517e+06 0.00e+00 1.69e+03  -8.6 2.69e+00    -  9.67e-01 9.97e-01f  1
  36  1.4042516e+06 0.00e+00

In [ ]:
# # Other ways to access the optimal values:
# mdl.x['HRTill', 'N51133RL0_6450_0000', 'oac'].value
#
# tol = 1e-6
# for b in mdl.BMPS:
#     for lmbda in mdl.LOADSRCS:
#         bval = mdl.x[b, 'N51133RL0_6450_0000', lmbda].value
#         if not not bval:
#             if abs(bval)>tol:
#                 print('(%s, %s): %d' % (b, lmbda, bval))